In [ ]:
import json
import pickle
import numpy as np
import pandas as pd
from heuristic import *
from utils import *
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
%matplotlib inline

def get_df_for_heuristic():
    df = pd.read_csv("/Users/ashutoshshukla/Desktop/ThreeStageModel/data/192_Scenario/Final_Input1.csv")
    directions = ["w", "wnw", "nw", "nnw", "n", "nne"]
    categories = ["2", "3", "4", "5"]
    forward_speeds = ["05", "10", "15", "25"]
    lister = []
    for i in directions:
        for j in range(len(categories)):
            for k in range(len(forward_speeds)):
                lister.append("max_flood_level_" + i +"_" + categories[j] + "_" + forward_speeds[k])
    df = df[list(df.columns[0:9]) + lister]
    df_sub = df[["SubNum", "load"]].groupby("SubNum").sum()
    df_flood = df[["SubNum"] + lister]
    df_flood = df_flood.drop_duplicates().set_index("SubNum") # drop duplicates
    df_flood = df_flood.loc[(df_flood.sum(axis=1) != 0), :] # drop substations that are not flooded
    """df_sub has load demand for all the substations"""
    """df_flood has only flooded substations"""
    return df, df_sub, df_flood

def return_model_scenarios():
    directions = ["w", "wnw", "nw", "nnw", "n", "nne"]
    categories = ["2", "3", "4", "5"]
    forward_speeds = ["05", "10", "15", "25"]
    model_scenarios = {}
    counter = 0
    for i in directions:
        for j in range(len(categories)-1):
            for k in range(len(forward_speeds)-1):
                lister = []
                lister.append("max_flood_level_" + i +"_" + categories[j] + "_" + forward_speeds[k])
                lister.append("max_flood_level_" + i +"_" + categories[j] + "_" + forward_speeds[k+1])
                #lister.append("max_flood_level_" + i +"_" + categories[j] + "_" + forward_speeds[k+2])
                lister.append("max_flood_level_" + i +"_" + categories[j+1] + "_" + forward_speeds[k])
                lister.append("max_flood_level_" + i +"_" + categories[j+1] + "_" + forward_speeds[k+1])
                #lister.append("max_flood_level_" + i +"_" + categories[j+1] + "_" + forward_speeds[k+2])
                model_scenarios[counter] = lister
                counter = counter + 1
    return model_scenarios

def cmap_discretize(cmap, N):
    """Return a discrete colormap from the continuous colormap cmap.

        cmap: colormap instance, eg. cm.jet. 
        N: number of colors.

    Example
        x = resize(arange(100), (5,100))
        djet = cmap_discretize(cm.jet, 5)
        imshow(x, cmap=djet)
    """

    if type(cmap) == str:
        cmap = plt.get_cmap(cmap)
    colors_i = np.concatenate((np.linspace(0, 1., N), (0.,0.,0.,0.)))
    colors_rgba = cmap(colors_i)
    indices = np.linspace(0, 1., N+1)
    cdict = {}
    for ki,key in enumerate(('red','green','blue')):
        cdict[key] = [ (indices[i], colors_rgba[i-1,ki], colors_rgba[i,ki])
                       for i in range(N+1) ]
    # Return colormap object.
    return mcolors.LinearSegmentedColormap(cmap.name + "_%d"%N, cdict, 1024)

df, df_sub, df_flood = get_df_for_heuristic()
model_scenarios = return_model_scenarios()

with open('optimization_solution_list.pickle', 'rb') as handle:
    optimization_list = pickle.load(handle)

In [1]:
! git branch

* final
  main
  paper


In [ ]:
voll = 6000
df_plot = pd.DataFrame(index=df_flood.index, columns=np.arange(0,54,1))
lister = []
for i in df_plot.index:
    for j in df_plot.columns:
        temp = str(i) + "_" + str(j)
        td_value = optimization_list[voll][1][temp]
        flood = df_flood.loc[i, model_scenarios[j]].max()
        if td_value == 0:
            if flood == 0:
                df_plot.loc[i,j] = 0.0
            else:
                df_plot.loc[i,j] = 1.0
        else:
            if td_value > flood:
                df_plot.loc[i,j] = 2.0
            elif td_value == flood:
                df_plot.loc[i,j] = 3.0
            else:
                df_plot.loc[i,j] = 4.0
    if optimization_list[voll][0][i] > 0:
        lister.append(5.0)
    else:
        lister.append(6.0)
    
df_plot["hardening"] = lister
df_plot["hardening1"] = lister
df_plot["hardening2"] = lister
df_plot["hardening3"] = lister

colormap_mapping = {
    0.0: 'red',
    1.0: 'green',
    2.0: 'blue',
    3.0: 'purple',
    4.0: 'orange',
    5.0: "black",
    6.0: "yellow"
}
colors = [colormap_mapping[i] for i in range(7)]
cmap = mpl.colors.ListedColormap(colors)

plt.figure(figsize=(10,5))
plt.matshow(df_plot.iloc[:,:].values.astype(float),cmap=cmap)
cb = plt.colorbar(shrink=0.8)
labels = np.arange(0,7, 0.2)
loc    = [0.45,1.3,2.1,3,3.8,4.7,5.6]
cb.set_ticks(loc)
l1 = ["td = max_flood = 0", "td = 0 < max_flood", "td > max_flood > 0", "td = max_flood > 0",
      "0 <td < max_flood", "hardened", "not hardened"]
      
cb.set_ticklabels(l1)
plt.tick_params(which = 'both', size = 0, labelsize = 0)
plt.tight_layout()
plt.xlabel("Preparedness Models")
plt.ylabel("Substations")
#plt.savefig("output_plots/voll_variation_" + str(voll) + ".pdf", format="pdf", bbox_inches="tight")
plt.show()

# Flood Map

In [ ]:
plt.figure(figsize=(20,20))
plt.matshow(df_flood.iloc[:,:].values.astype(float))
cb = plt.colorbar(shrink=0.8)

#plt.savefig("output_plots/flooding.pdf", format="pdf", bbox_inches="tight")
plt.show()

# General method for any kind of run

In [ ]:
model_name_list = [250,500,1000,2000,3000,4000,5000,6000]

for voll_c in model_name_list:
    print(voll_c)
    model_name = "modified_td_coordination_" + str(voll_c)
    main_path = "/Users/ashutoshshukla/Downloads/" + model_name + "/"

    with open(main_path + "model_params.json", 'r') as f:
        params = json.load(f)   
    params["path_to_input"] = os.getcwd() + "/data/192_Scenario/"

    with open(main_path + "model_scenarios.json", 'r') as f:
        model_scenarios_string = json.load(f)

    model_scenarios = {}
    for k in model_scenarios_string:
        model_scenarios[int(k)] = model_scenarios_string[k]

    base_model = three_stage_model(params, model_scenarios)
    base_model.model.update()
    sol_path = main_path + "solution.sol"
    base_model.model.read(sol_path)
    base_model.model.update()

    hardening_decisions = {}
    tiger_dam_decisions = {}

    for sub_id in df_flood.index:
        hardening_decisions[sub_id] = int(base_model.x_mit[sub_id].Start*params["mit_level"])
    for sub_id in df_flood.index:
        for j in model_scenarios:
            tiger_dam_decisions[str(sub_id) + "_" + str(j)] = int(base_model.x_prep[sub_id,j].Start*params["prep_level"])


    df_plot = pd.DataFrame(index=df_flood.index, columns=np.arange(0,54,1))
    lister = []
    for i in df_plot.index:
        for j in df_plot.columns:
            temp = str(i) + "_" + str(j)
            td_value = tiger_dam_decisions[temp]
            flood = df_flood.loc[i, model_scenarios[j]].max()
            if td_value == 0:
                if flood == 0:
                    df_plot.loc[i,j] = 0.0
                else:
                    df_plot.loc[i,j] = 1.0
            else:
                if td_value > flood:
                    df_plot.loc[i,j] = 2.0
                elif td_value == flood:
                    df_plot.loc[i,j] = 3.0
                else:
                    df_plot.loc[i,j] = 4.0
        if hardening_decisions[i] > 0:
            lister.append(5.0)
        else:
            lister.append(6.0)

    df_plot["hardening"] = lister
    df_plot["hardening1"] = lister
    df_plot["hardening2"] = lister
    df_plot["hardening3"] = lister

    colormap_mapping = {
        0.0: 'red',
        1.0: 'green',
        2.0: 'blue',
        3.0: 'purple',
        4.0: 'orange',
        5.0: "black",
        6.0: "yellow"
    }
    colors = [colormap_mapping[i] for i in range(7)]
    cmap = mpl.colors.ListedColormap(colors)

    plt.figure(figsize=(10,5))
    plt.matshow(df_plot.iloc[:,:].values.astype(float),cmap=cmap)
    cb = plt.colorbar(shrink=0.8)
    labels = np.arange(0,7, 0.2)
    loc    = [0.45,1.3,2.1,3,3.8,4.7,5.6]
    cb.set_ticks(loc)
    l1 = ["td = max_flood = 0", "td = 0 < max_flood", "td > max_flood > 0", "td = max_flood > 0",
          "0 <td < max_flood", "hardened", "not hardened"]

    cb.set_ticklabels(l1)
    plt.tick_params(which = 'both', size = 0, labelsize = 0)
    plt.tight_layout()
    plt.xlabel("Preparedness Models")
    plt.ylabel("Substations")
    plt.savefig("output_plots/" + model_name + ".pdf", format="pdf", bbox_inches="tight")
    #plt.show()

# Solution Analysis

In [ ]:
def model_analysis(main_path):    
    with open(main_path + "model_params.json", 'r') as f:
        params = json.load(f)   
    params["path_to_input"] = os.getcwd() + "/data/192_Scenario/"
    with open(main_path + "model_scenarios.json", 'r') as f:
        model_scenarios_string = json.load(f)
    model_scenarios = {}
    for k in model_scenarios_string:
        model_scenarios[int(k)] = model_scenarios_string[k]
    base_model = three_stage_model(params, model_scenarios)
    base_model.model.update()
    sol_path = main_path + "solution.sol"
    base_model.model.read(sol_path)
    base_model.model.update()

    hardening_decisions = {}
    tiger_dam_decisions = {}

    for sub_id in df_flood.index:
        hardening_decisions[sub_id] = int(base_model.x_mit[sub_id].Start*params["mit_level"])
    for sub_id in df_flood.index:
        for j in model_scenarios:
            tiger_dam_decisions[str(sub_id) + "_" + str(j)] = int(base_model.x_prep[sub_id,j].Start*params["prep_level"])
    
    return hardening_decisions, tiger_dam_decisions, base_model

In [ ]:
main_path = "/Users/ashutoshshukla/Downloads/modified_flexible_mit_24/"
hard_18, td_18, base_18 = model_analysis(main_path)

In [ ]:
main_path = "/Users/ashutoshshukla/Downloads/modified_td_voll_6000/"
hard, td, base = model_analysis(main_path)

In [ ]:
main_lister = []

for i in hard_18:
    main_lister.append([hard_18[i], hard[i]])

In [ ]:
df1 = pd.DataFrame(main_lister)

In [ ]:
df = pd.DataFrame(main_lister)

In [ ]:
df1["round"] = df[0]

In [ ]:
df1[df1["round"] > 0].shape

In [ ]:
df_flood.loc[[327, 929, 1025, 1084, 1104], :].max(axis=1)

In [ ]:
df1[df1[1] > df1["round"]][[1, "round"]]

# 8 instances where coarse model did not harden substation which implies here we will lose power 

In [ ]:
df_sub.loc[[327, 349, 929, 952, 979, 1025, 1084, 1104],:]

In [ ]:
df1.index = list(hardening_decisions.keys())

df2 = df1[df1[1] < df1["round"]][[1, "round"]]

pd.Series(df2["round"] - df[1]).sum()

* 29 substations are over hardened. Here we put $9.7M extra. But this overhardening doesn't give us any benefit. However under-hardening can hurt us.

* 8 substations are under hardened. Out of these 5 substations were not hardened at all. Now, if flooding at these substations in some of the cases is less than 6 feet, we will protect them using tiger dams. This requires buying extra tiger dams. Infact model purchases 24 extra tiger dam units which is reflected in extra tiger dam cost. Two out of these 5 substations have max flood level > 6 feet and therefore they also lead to extra load shed. For other 3 substations, the hardening level of the non-binary model is 12 feet whereas their max flood level is 13,15, and 16 feet. So even in this case, there is extra load shed. 

* If all these 8 substations are hardened to avoid load shed, it will cost 4.5M but, we loss net power worth 2.7M. So, that is why solution is what it is.

* 35 have same hardening level. So nothing to think about here.